In [1]:
import pandas as pd
import tensorflow_hub as hub
import openai 
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
import ast
import cx_Oracle
import re
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import spacy
import nltk 
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import unicodedata

from pathlib import Path
import sys
sys.path.append('../')
import scripts.eval
from scripts.eval import execute_query 
from scripts.eval import metric 
import json
import requests
import datetime
nltk.download('stopwords')
nlp = spacy.load('en_core_web_lg')
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

2024-06-11 08:46:23.607528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 08:46:23.607596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 08:46:23.628386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 08:46:23.671384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 08:46:24.415236: W tensorflow/compiler/tf2

ModuleNotFoundError: No module named 'src'

In [2]:
prefix = Path.cwd().parent.parent
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
prefix

PosixPath('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise')

In [8]:
df=pd.read_json(str(prefix) + '/data/spider/dev.json')
db_t = {}
for i in range(len(df)):
    if df['db_id'][i] in db_t.keys():
        db_t[df['db_id'][i]] = db_t[df['db_id'][i]] | set(Parser(df['query'][i]).tables)
    else:
        db_t[df['db_id'][i]]=set(Parser(df['query'][i]).tables)
db_id = {}
for db in df['db_id']:
    if db in db_id.keys():
        db_id[db] += 1
    else:
        db_id[db]=1
for db in db_id:
    if db_id[db] >50:
        print("'{0}': {1} : {2}".format(db, db_id[db], len(db_t[db])))

'car_1': 92 : 12
'flight_2': 80 : 5
'cre_Doc_Template_Mgt': 84 : 5
'wta_1': 62 : 3
'student_transcripts_tracking': 78 : 15
'tvshow': 62 : 5
'world_1': 120 : 3
'network_1': 56 : 3
'dog_kennels': 82 : 11


In [9]:
df_sample = df[df['db_id']=="student_transcripts_tracking"].sample(n=50)

In [18]:
possible = set(df[df['db_id']=="student_transcripts_tracking"].index).difference(set(df_sample.index))
questions = []
sqls = []
for i in possible:
    questions.append(df['question'][i])
    sqls.append(df['query'][i])
df_examples = pd.DataFrame(data = {'question':questions, 'sql':sqls})

In [21]:

df_sample1 = pd.DataFrame(data = {'question':df_sample['question'], 'sql':df_sample['query']})

In [22]:
#df_example = df_examples.sample(n=5)
#df_example.to_json(str(prefix)+'/data/spider/examples.json', index=False, orient='records')
df_sample1.to_json(str(prefix)+'/data/spider/queries.json', index=False, orient='records')

In [19]:

df[df['db_id']=="student_transcripts_tracking"].sample(n=50).to_json(str(prefix)+'/data/test.json', index=False, orient='records')

In [4]:
df=pd.read_json(str(prefix) + '/data/spider/tables.json')


In [20]:
print('COLUMN_NAME,DATA_TYPE')
for i in range(len(df['column_types'][79])):
    if df['column_names_original'][79][i][0] == 11:
        print(df['column_names_original'][79][i][1] + ',' + df['column_types'][79][i].upper())

COLUMN_NAME,DATA_TYPE


In [6]:
df[df['db_id']=='student_transcripts_tracking']
tables ={}
for 


,column_names,column_names_original,column_types,db_id,foreign_keys,primary_keys,table_names,table_names_original
79,"[[-1, *], [0, address id], [0, line 1], [0, line 2], [0, line 3], [0, city], [0, zip postcode], [0, state province county], [0, country], [0, other address details], [1, course id], [1, course name], [1, course description], [1, other details], [2, department id], [2, department name], [2, department description], [2, other details], [3, degree program id], [3, department id], [3, degree summary name], [3, degree summary description], [3, other details], [4, section id], [4, course id], [4, section name], [4, section description], [4, other details], [5, semester id], [5, semester name], [5, semester description], [5, other details], [6, student id], [6, current address id], [6, permanent address id], [6, first name], [6, middle name], [6, last name], [6, cell mobile number], [6, email address], [6, ssn], [6, date first registered], [6, date left], [6, other student details], [7, student enrolment id], [7, degree program id], [7, semester id], [7, student id], [7, other details], [8, student course id], [8, course id], [8, student enrolment id], [9, transcript id], [9, transcript date], [9, other details], [10, student course id], [10, transcript id]]","[[-1, *], [0, address_id], [0, line_1], [0, line_2], [0, line_3], [0, city], [0, zip_postcode], [0, state_province_county], [0, country], [0, other_address_details], [1, course_id], [1, course_name], [1, course_description], [1, other_details], [2, department_id], [2, department_name], [2, department_description], [2, other_details], [3, degree_program_id], [3, department_id], [3, degree_summary_name], [3, degree_summary_description], [3, other_details], [4, section_id], [4, course_id], [4, section_name], [4, section_description], [4, other_details], [5, semester_id], [5, semester_name], [5, semester_description], [5, other_details], [6, student_id], [6, current_address_id], [6, permanent_address_id], [6, first_name], [6, middle_name], [6, last_name], [6, cell_mobile_number], [6, email_address], [6, ssn], [6, date_first_registered], [6, date_left], [6, other_student_details], [7, student_enrolment_id], [7, degree_program_id], [7, semester_id], [7, student_id], [7, other_details], [8, student_course_id], [8, course_id], [8, student_enrolment_id], [9, transcript_id], [9, transcript_date], [9, other_details], [10, student_course_id], [10, transcript_id]]","[text, number, text, text, text, text, text, text, text, text, number, text, text, text, number, text, text, text, number, number, text, text, text, number, number, text, text, text, number, text, text, text, number, number, number, text, text, text, text, text, text, time, time, text, number, number, number, number, text, number, number, number, number, time, text, number, number]",student_transcripts_tracking,"[[19, 14], [24, 10], [34, 1], [33, 1], [47, 32], [46, 28], [45, 18], [51, 44], [50, 10], [56, 52], [55, 49]]","[1, 10, 14, 18, 23, 28, 32, 44, 49, 52]","[addresses, courses, departments, degree programs, sections, semesters, students, student enrolment, student enrolment courses, transcripts, transcript contents]","[Addresses, Courses, Departments, Degree_Programs, Sections, Semesters, Students, Student_Enrolment, Student_Enrolment_Courses, Transcripts, Transcript_Contents]"


In [6]:
prefix = Path.cwd().parent.parent
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
prefix
with open('../../config.json') as f:
  config = json.load(f)

# Read the MIT DW queries 

In [8]:
df_q = pd.read_parquet(str(prefix) + '/data/train-00000-of-00001.parquet')

In [9]:
df_q

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older than 56 ?,"[SELECT, count, (, *, ), FROM, head, WHERE, age, >, 56]","[select, count, (, *, ), from, head, where, age, >, value]","[How, many, heads, of, the, departments, are, older, than, 56, ?]"
1,department_management,"SELECT name , born_state , age FROM head ORDER BY age","List the name, born state and age of the heads of departments ordered by age.","[SELECT, name, ,, born_state, ,, age, FROM, head, ORDER, BY, age]","[select, name, ,, born_state, ,, age, from, head, order, by, age]","[List, the, name, ,, born, state, and, age, of, the, heads, of, departments, ordered, by, age, .]"
2,department_management,"SELECT creation , name , budget_in_billions FROM department","List the creation year, name and budget of each department.","[SELECT, creation, ,, name, ,, budget_in_billions, FROM, department]","[select, creation, ,, name, ,, budget_in_billions, from, department]","[List, the, creation, year, ,, name, and, budget, of, each, department, .]"
3,department_management,"SELECT max(budget_in_billions) , min(budget_in_billions) FROM department",What are the maximum and minimum budget of the departments?,"[SELECT, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), FROM, department]","[select, max, (, budget_in_billions, ), ,, min, (, budget_in_billions, ), from, department]","[What, are, the, maximum, and, minimum, budget, of, the, departments, ?]"
4,department_management,SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,What is the average number of employees of the departments whose rank is between 10 and 15?,"[SELECT, avg, (, num_employees, ), FROM, department, WHERE, ranking, BETWEEN, 10, AND, 15]","[select, avg, (, num_employees, ), from, department, where, ranking, between, value, and, value]","[What, is, the, average, number, of, employees, of, the, departments, whose, rank, is, between, 10, and, 15, ?]"
...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS T1 JOIN book_club AS T2 ON T1.book_club_id = T2.book_club_id WHERE T2.publisher = 'Alyson',What are all the company names that have a book published by Alyson?,"[SELECT, T1.company_name, FROM, culture_company, AS, T1, JOIN, book_club, AS, T2, ON, T1.book_club_id, =, T2.book_club_id, WHERE, T2.publisher, =, 'Alyson, ']","[select, t1, ., company_name, from, culture_company, as, t1, join, book_club, as, t2, on, t1, ., book_club_id, =, t2, ., book_club_id, where, t2, ., publisher, =, value]","[What, are, all, the, company, names, that, have, a, book, published, by, Alyson, ?]"
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id = T2.movie_id JOIN book_club AS T3 ON T3.book_club_id = T2.book_club_id WHERE T2.incorporated_in = 'China'",Show the movie titles and book titles for all companies in China.,"[SELECT, T1.title, ,, T3.book_title, FROM, movie, AS, T1, JOIN, culture_company, AS, T2, ON, T1.movie_id, =, T2.movie_id, JOIN, book_club, AS, T3, ON, T3.book_club_id, =, T2.book_club_id, WHERE, T2.incorporated_in, =, 'China, ']","[select, t1, ., title, ,, t3, ., book_title, from, movie, as, t1, join, culture_company, as, t2, on, t1, ., movie_id, =, t2, ., movie_id, join, book_club, as, t3, on, t3, ., book_club_id, =, t2, ., book_club_id, where, t2, ., incorporated_in, =, value]","[Show, the, movie, titles, and, book, titles, for, all, companies, in, China, .]"
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id = T2.movie_id JOIN book_club AS T3 ON T3.book_club_id = T2.book_club_id WHERE T2.incorporated_in = 'China'",What are the titles of movies and books corresponding to companies incorporated in China?,"[SELECT, T1.title, ,, T3.book_title, FROM, movie, AS, T1, JOIN, culture_company, AS, T2, ON, T1.movie_

In [10]:
unique_ids = set()
for  l in df_q['db_id']: 
    unique_ids.add(l)

In [11]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [13]:

# Set the random seed
random.seed(42)

value_counts = df_q['db_id'].value_counts()

# Display the counts
print(value_counts)

# Assuming 'value_counts' is the Series containing value counts
filtered_value_counts = value_counts[value_counts > 50]

# Choose a random element from the filtered Series
random_element = random.choice(filtered_value_counts.index)
print(random_element)

db_id
college_2                 170
college_1                 164
hr_1                      124
store_1                   112
soccer_2                  106
                         ... 
solvency_ii                15
decoration_competition     15
soccer_1                   14
local_govt_mdm             14
company_1                   7
Name: count, Length: 140, dtype: int64
csu_1


In [14]:
(df_q['db_id'] == random_element).sum()

70

In [15]:

# Assuming 'df_q' is your DataFrame
subset_with_dorm_1 = df_q[df_q['db_id'] == random_element]

# Take a random sample from the subset
df_q = subset_with_dorm_1.sample(n=50, random_state=42) 
df_q = df_q.reset_index(drop=True)
df_q

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,csu_1,SELECT YEAR FROM degrees GROUP BY YEAR ORDER BY sum(degrees) DESC LIMIT 1,Which year has the most degrees conferred?,"[SELECT, YEAR, FROM, degrees, GROUP, BY, YEAR, ORDER, BY, sum, (, degrees, ), DESC, LIMIT, 1]","[select, year, from, degrees, group, by, year, order, by, sum, (, degrees, ), desc, limit, value]","[Which, year, has, the, most, degrees, conferred, ?]"
1,csu_1,"SELECT campus FROM campuses WHERE county = ""Los Angeles""",Report the name of all campuses in Los Angeles county.,"[SELECT, campus, FROM, campuses, WHERE, county, =, ``, Los, Angeles, '']","[select, campus, from, campuses, where, county, =, value]","[Report, the, name, of, all, campuses, in, Los, Angeles, county, .]"
2,csu_1,SELECT sum(faculty) FROM faculty WHERE YEAR = 2002,"How many faculty, in total, are there in the year 2002?","[SELECT, sum, (, faculty, ), FROM, faculty, WHERE, YEAR, =, 2002]","[select, sum, (, faculty, ), from, faculty, where, year, =, value]","[How, many, faculty, ,, in, total, ,, are, there, in, the, year, 2002, ?]"
3,csu_1,SELECT campus FROM campuses WHERE YEAR = 1958,Find all the campuses opened in 1958.,"[SELECT, campus, FROM, campuses, WHERE, YEAR, =, 1958]","[select, campus, from, campuses, where, year, =, value]","[Find, all, the, campuses, opened, in, 1958, .]"
4,csu_1,SELECT T1.campus FROM campuses AS t1 JOIN faculty AS t2 ON t1.id = t2.campus WHERE t2.faculty >= 600 AND t2.faculty <= 1000 AND T1.year = 2004,List the campus that have between 600 and 1000 faculty lines in year 2004.,"[SELECT, T1.campus, FROM, campuses, AS, t1, JOIN, faculty, AS, t2, ON, t1.id, =, t2.campus, WHERE, t2.faculty, >, =, 600, AND, t2.faculty, <, =, 1000, AND, T1.year, =, 2004]","[select, t1, ., campus, from, campuses, as, t1, join, faculty, as, t2, on, t1, ., id, =, t2, ., campus, where, t2, ., faculty, >, =, value, and, t2, ., faculty, <, =, value, and, t1, ., year, =, value]","[List, the, campus, that, have, between, 600, and, 1000, faculty, lines, in, year, 2004, .]"
...,...,...,...,...,...,...
45,csu_1,"SELECT campusfee FROM campuses AS T1 JOIN csu_fees AS T2 ON T1.id = t2.campus WHERE t1.campus = ""San Francisco State University"" AND T2.year = 1996",What is the campus fee for San Francisco State University in 1996?,"[SELECT, campusfee, FROM, campuses, AS, T1, JOIN, csu_fees, AS, T2, ON, T1.id, =, t2.campus, WHERE, t1.campus, =, ``, San, Francisco, State, University, '', AND, T2.year, =, 1996]","[select, campusfee, from, campuses, as, t1, join, csu_fees, as, t2, on, t1, ., id, =, t2, ., campus, where, t1, ., campus, =, value, and, t2, ., year, =, value]","[What, is, the, campus, fee, for, San, Francisco, State, University, in, 1996, ?]"
46,csu_1,SELECT T1.campus FROM campuses AS T1 JOIN faculty AS T2 ON T1.id = T2.campus WHERE T2.year = 2003 ORDER BY T2.faculty DESC LIMIT 1,What campus has the most faculties in 2003?,"[SELECT, T1.campus, FROM, campuses, AS, T1, JOIN, faculty, AS, T2, ON, T1.id, =, T2.campus, WHERE, T2.year, =, 2003, ORDER, BY, T2.faculty, DESC, LIMIT, 1]","[select, t1, ., campus, from, campuses, as, t1, join, faculty, as, t2, on, t1, ., id, =, t2, ., campus, where, t2, ., year, =, value, order, by, t2, ., faculty, desc, limit, value]","[What, campus, has, the, most, faculties, in, 2003, ?]"
47,csu_1,"SELECT count(*) FROM campuses WHERE county = ""Los Angeles""",How many campuses exist are in the county of LA?,"[SELECT, count, (, *, ), FROM, campuses, WHERE, county, =, ``, Los, Angeles, '']","[select, count, (, *, ), from, campuses, where, county, =, value]","[How, many, campuses, exist, are, in, the, county, of, LA, ?]"
48,csu_1,SELECT T1.campus FROM campuses AS T1 JOIN faculty AS T2 ON T1.id = T2.campus WHERE T2.year = 2003 ORDER BY T2.faculty DESC LIMIT 1,Which campus has the most faculties in year 2003?,"[SELECT, T1.campus, FROM, campuses, AS, T1, JOIN, faculty, AS, T2, ON, T1.id, =, T2.campus, WHERE, T2.year, =, 2003, ORDER, BY, T2.faculty, DESC, L

## Identify the tables used by each query 

In [16]:
tables = []
for i in range(len(df_q)):
    table = []
    t = Parser(df_q['query'][i]).tables
    for v in t:
        table.append(v.upper())
    tables.append(table)
df_q['tables'] = tables

## Identify the unique tables referred by the corpus 

In [17]:
unique_tables = set()
for  l in df_q['tables']: 
    for t in l:
        unique_tables.add(t.upper())

In [18]:
unique_tables

{'CAMPUSES',
 'CSU_FEES',
 'DEGREES',
 'DISCIPLINE_ENROLLMENTS',
 'ENROLLMENTS',
 'FACULTY'}

In [19]:
len(unique_tables)

6

## Go thrugh the `unique_tables` list and save the respective schemas as proper CSV files 

In [33]:
#for t in unique_tables: 
#    if t.upper().startswith('WAREUSER.'):
#        t = t[len('WAREUSER.'):]
#    schema_file = prefix + 'schema/' + t + '.csv'
#    if os.path.isfile(schema_file): 
#        s = pd.read_csv(schema_file, header=None)
#        s = s.drop(1) # delete the "header" line 
#        column_names = [h.strip() for h in s[0][0].split(';')]
#        s_new = pd.DataFrame(list(s[0][1:].apply(lambda row: [c.strip() for c in row.split(';')])), columns=column_names)
#        new_schema_file = prefix + 'schema_new/' + t + '.csv'
#        s_new.to_csv(new_schema_file, index=False)

# Treat ChatGPT as an all-knowing oracle (without any additional information)
## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at 
MIT, containing information about students, faculty, and personnel. You can find more about the MIT data 
warehouse tables at https://web.mit.edu/warehouse/metadata/tables/all_tables.html._

_How many organisation units in human resources are relabeled?_

In [35]:
client = OpenAI(api_key=api_key)

## Populate the prompt and call the Open AI API

In [81]:
import requests
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

response = requests.get('https://api.openai.com/v1/engines', headers=headers)

if response.status_code == 200:
    engines = response.json()['data']
    print("Available engines:")
    for engine in engines:
        print(engine['id'])
else:
    print("Error:", response.text)

Available engines:
whisper-1
davinci-002
gpt-4-1106-preview
gpt-4-0613
gpt-4-turbo
dall-e-2
tts-1-hd-1106
tts-1-hd
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-4
gpt-4-1106-vision-preview
gpt-3.5-turbo-0125
gpt-4-turbo-2024-04-09
gpt-3.5-turbo
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
tts-1
dall-e-3
gpt-3.5-turbo-0613
gpt-3.5-turbo-0301
gpt-3.5-turbo-instruct-0914
babbage-002
gpt-3.5-turbo-instruct
tts-1-1106
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
gpt-3.5-turbo-1106
gpt-4-vision-preview


In [83]:
# TODO: do this in batch 
gpt_sql_nocontext = []
for q in df_q['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the dorm_1 database\
          in the Spider Dataset. \
          The dorm_1 dataset likely comes from a specific data source used to create the Spider dataset. The Spider dataset \
          is a collection of question-answer pairs based on databases, and 'dorm' is representing one source of these \
          databases on which question-answers pairs in the spider-dataset exist. \
          Reply with as many answers \
          as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
          answer, but seperate the possibilities by semicolons."
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
          },
            {"role":"user", 
             "content":q}]
    )
    gpt_sql_nocontext.append(response.choices[0].message.content)
    print(response.choices[0].message.content)

SELECT Amenities.amenity FROM Amenities JOIN Dorms ON Amenities.dorm_id = Dorms.dorm_id WHERE Dorms.name = 'Smith Hall'; SELECT amenity FROM Amenities WHERE dorm_id = (SELECT dorm_id FROM Dorms WHERE name = 'Smith Hall')
SELECT COUNT(*), gender, SUM(capacity) FROM dorm GROUP BY gender;
SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name; SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms LEFT JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name;
SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1;SELECT city_code FROM student WHERE student_id IN (SELECT student_id FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1);
SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1;
SELECT student.LastName FROM student 

In [85]:
df_q['gpt_sql_nocontext']=gpt_sql_nocontext

In [86]:
df_q

,db_id,query,question,query_toks,query_toks_no_value,question_toks,tables,gpt_sql_nocontext
0,dorm_1,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',What are the names of the amenities that Smith Hall has?,"[SELECT, T3.amenity_name, FROM, dorm, AS, T1, JOIN, has_amenity, AS, T2, ON, T1.dormid, =, T2.dormid, JOIN, dorm_amenity, AS, T3, ON, T2.amenid, =, T3.amenid, WHERE, T1.dorm_name, =, 'Smith, Hall, ']","[select, t3, ., amenity_name, from, dorm, as, t1, join, has_amenity, as, t2, on, t1, ., dormid, =, t2, ., dormid, join, dorm_amenity, as, t3, on, t2, ., amenid, =, t3, ., amenid, where, t1, ., dorm_name, =, value]","[What, are, the, names, of, the, amenities, that, Smith, Hall, has, ?]","[DORM, HAS_AMENITY, DORM_AMENITY]",SELECT Amenities.amenity FROM Amenities JOIN Dorms ON Amenities.dorm_id = Dorms.dorm_id WHERE Dorms.name = 'Smith Hall'; SELECT amenity FROM Amenities WHERE dorm_id = (SELECT dorm_id FROM Dorms WHERE name = 'Smith Hall')
1,dorm_1,"SELECT count(*) , sum(student_capacity) , gender FROM dorm GROUP BY gender",How many dorms are there and what is the total capacity for each gender?,"[SELECT, count, (, *, ), ,, sum, (, student_capacity, ), ,, gender, FROM, dorm, GROUP, BY, gender]","[select, count, (, *, ), ,, sum, (, student_capacity, ), ,, gender, from, dorm, group, by, gender]","[How, many, dorms, are, there, and, what, is, the, total, capacity, for, each, gender, ?]",[DORM],"SELECT COUNT(*), gender, SUM(capacity) FROM dorm GROUP BY gender;"
2,dorm_1,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid",Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"[SELECT, count, (, *, ), ,, T1.dormid, FROM, dorm, AS, T1, JOIN, has_amenity, AS, T2, ON, T1.dormid, =, T2.dormid, WHERE, T1.student_capacity, >, 100, GROUP, BY, T1.dormid]","[select, count, (, *, ), ,, t1, ., dormid, from, dorm, as, t1, join, has_amenity, as, t2, on, t1, ., dormid, =, t2, ., dormid, where, t1, ., student_capacity, >, value, group, by, t1, ., dormid]","[Find, the, number, of, amenities, for, each, of, the, dorms, that, can, accommodate, more, than, 100, students, .]","[DORM, HAS_AMENITY]","SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name; SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms LEFT JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name;"
3,dorm_1,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,What is the code of the city with the most students?,"[SELECT, city_code, FROM, student, GROUP, BY, city_code, ORDER, BY, count, (, *, ), DESC, LIMIT, 1]","[select, city_code, from, student, group, by, city_code, order, by, count, (, *, ), desc, limit, value]","[What, is, the, code, of, the, city, with, the, most, students, ?]",[STUDENT],SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1;SELECT city_code FROM student WHERE student_id IN (SELECT student_id FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1);
4,dorm_1,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,Find the code of city where most of students are living in.,"[SELECT, city_code, FROM, student, GROUP, BY, city_code, ORDER, BY, count, (, *, ), DESC, LIMIT, 1]","[select, city_code, from, student, group, by, city_code, order, by, count, (, *, ), desc, limit, value]","[Find, the, code, of, city, where, most, of, students, are, living, in, .]",[STUDENT],SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1;
...,...,...,...,...,...,...,...,...
45,dorm_1,"SELECT avg(age) , city_code FROM stu

In [88]:
sql = []
for q in gpt_sql_nocontext:
    sql.append(q.split(';'))

In [89]:
sqls = []
for sq in sql:
    sqls_=[]
    for s in sq:
        if s != '':
            sqls_.append(s)
    sqls.append(sqls_)
sqls

[["SELECT Amenities.amenity FROM Amenities JOIN Dorms ON Amenities.dorm_id = Dorms.dorm_id WHERE Dorms.name = 'Smith Hall'",
  " SELECT amenity FROM Amenities WHERE dorm_id = (SELECT dorm_id FROM Dorms WHERE name = 'Smith Hall')"],
 ['SELECT COUNT(*), gender, SUM(capacity) FROM dorm GROUP BY gender'],
 ['SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name',
  ' SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms LEFT JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name'],
 ['SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1',
  'SELECT city_code FROM student WHERE student_id IN (SELECT student_id FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1)'],
 ['SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1

In [90]:
df_q['gpt_sql_nocontext']=sqls

# Give ChatGPT the "database" schema as context and elicit its answer


In [40]:
def table_to_sql_create(t): 
    schema_file=prefix + 'schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE "+ t+ "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
    return p
    

#### Generalize table-names (delete "WAREUSER." in front) and create the prefix for the prompt.


In [41]:
prompt = "** all the tables in the database ** \n\n"
cnt = 0 
for t in unique_tables:
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    p = table_to_sql_create(t)
    if not p: 
        print(t)
    if p: 
        cnt = cnt + 1
        prompt = prompt + p
        prompt = prompt + "\n\n"


#### There should be no tables listed above (only if the table-name does not occur in our extracted table-schemas).

In [42]:
## The "schema" prompt, which we append to the system prompt below 
print(prompt)

** all the tables in the database ** 

CREATE TABLE TOP_LEVEL_DOMAIN(
TOP_LEVEL_DOMAIN_KEY VARCHAR2
TOP_LEVEL_DOMAIN VARCHAR2
TOP_LEVEL_DOMAIN_DESCRIPTION VARCHAR2
WAREHOUSE_LOAD_DATE DATE)


CREATE TABLE FCLT_MAJOR_USE(
FCLT_MAJOR_USE_KEY VARCHAR2
MAJOR_USE VARCHAR2
DESCRIPTION VARCHAR2
ASSIGNABLE VARCHAR2
WAREHOUSE_LOAD_DATE DATE)


CREATE TABLE ACADEMIC_TERM_PARAMETER(
TERM_PARAMETER VARCHAR2
TERM_INDICATOR VARCHAR2
TERM_CODE VARCHAR2
TERM_DESCRIPTION VARCHAR2
TERM_START_DATE DATE
TERM_END_DATE DATE
TERM_LAST_DAY_BEFORE_NEXT_TERM DATE
IS_CURRENT_TERM VARCHAR2)


CREATE TABLE HR_ORG_UNIT(
HR_ORG_UNIT_KEY VARCHAR2
HR_ORG_UNIT_ID VARCHAR2
HR_ORG_UNIT_TITLE VARCHAR2
HR_ORG_UNIT_LEVEL VARCHAR2
HR_DEPARTMENT_ID VARCHAR2
HR_DEPARTMENT_CODE VARCHAR2
HR_DEPARTMENT_CODE_OLD VARCHAR2
HR_DEPARTMENT_NAME VARCHAR2
HR_DEPARTMENT_NAME_LONG VARCHAR2
HR_DEPARTMENT_NAME_ALPHA VARCHAR2
ORG_HIER_SCHOOL_AREA_NAME VARCHAR2
ORG_HIER_TOP_LEVEL_NAME VARCHAR2
ORG_HIER_ROOT_NAME VARCHAR2
HR_ORG_LEVEL1_ID VARCH

## We're now ready to make the api call 

In [43]:
gpt_sql_global_schema = []
for q in df['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature = 0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of all the tables that you might need \
          for answering the user questions are below. There are 50 tables in the database.  Reply with as many answers \
           as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
           answer, but seperate the possibilities by semicolons. \n\n" + prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_global_schema.append(response.choices[0].message.content)
    print(response.choices[0].message.content)
df['gpt_sql_global_schema']=gpt_sql_global_schema
df.to_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/gpt_sql_v0.csv', index=False)

SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID, SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_CATALOG_SUBJECT_OFFERED GROUP BY DEPARTMENT_NAME
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE < CURRENT_DATE ORDER BY TERM_START_DATE DESC, SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE > CURRENT_DATE ORDER BY TERM_START_DATE ASC, SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE <= CURRENT_DATE AND TERM_END_DATE >= CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_Courses FROM SUBJECT_OFFERED
SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PERSON.MIT_ID FROM MIT_STUDENT_DIRECTORY I

In [44]:
df['gpt_sql_global_schema']=gpt_sql_global_schema

In [45]:
df

,question,sql,tables,gpt_sql_nocontext,gpt_sql_global_schema
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM hr_organization_units WHE...,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_...
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM human_resources WHERE is_...,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_...
2,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS COURSE_CO...","[CIS_COURSE_CATALOG, COURSE_CATALOG_SUBJECT_OF...","""SELECT department_id, COUNT(*) FROM courses G...","SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_C..."
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, EFFECTIVE_TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS, DRUPAL_CO...",SELECT * FROM courses WHERE course_date < CURR...,SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_...
4,Which faculty has had the most location changes?,[SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT ...,[FCLT_BUILDING_ADDRESS_HIST],"SELECT faculty_id, COUNT(DISTINCT location_id)...","SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFF..."
5,What is the average number of courses a studen...,[SELECT AVG(CNT) FROM (SELECT COUNT(*) AS CNT ...,"[MIT_STUDENT_DIRECTORY, CIS_COURSE_CATALOG, MI...",SELECT AVG(course_count) FROM (SELECT student_...,SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_C...
6,Which students are employed by mit?,[SELECT DISTINCT EMPL.FULL_NAME FROM MIT_STUDE...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY, MI...",SELECT student_id FROM students WHERE employme...,"SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PER..."
7,What is the shortest course available this sem...,"[SELECT DISTINCT SUBJECT_TITLE, TERM_END_DATE ...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG, ACADEMIC_...",SELECT MIN(course_length) FROM courses WHERE s...,"SELECT SUBJECT_ID, SUBJECT_TITLE, MIN(TERM_DUR..."
8,Show buildings and their address.,"[SELECT FB.BUILDING_NAME, FBA.* FROM FAC_BUILD...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]","SELECT building_name, building_address FROM bu...","SELECT FCLT_BUILDING.BUILDING_NAME, FAC_BUILDI..."
9,How many students do we have?,[SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY;],[MIT_STUDENT_DIRECTORY],"SELECT COUNT(*) FROM students, SELECT COUNT(st...",SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY


# Data Cleaning of the gerenated answers.


In [60]:
splitted_sql = []
for i in range(len(df)):
    splitted_sql.append(df['gpt_sql_global_schema'][i].split(';'))
    if '\n' in df['gpt_sql_global_schema'][i] or '\\' in df['gpt_sql_global_schema'][i] or '"' in df['gpt_sql_global_schema'][i][0]:
        print("Pay attention to other splitting elements which GPT could have used...")
df['gpt_sql_global_schema'] = splitted_sql

# Generated for all answers of GPT a seperate ROW


In [91]:
question = []
sql = []
tables = []
gpt_sql_nocontext = []
for i in range(len(df_q)):
    for j in range(len(df_q['gpt_sql_nocontext'][i])):
        gpt_sql_nocontext.append(df_q['gpt_sql_nocontext'][i][j])
        question.append(df_q['question'][i])
        sql.append(df_q['query'][i])
        tables.append(df_q['tables'][i])
data = {
    'question': question,
    'sql': sql,
    'tables': tables,
    'gpt_sql_nocontext': gpt_sql_nocontext
}
df = pd.DataFrame(data)
df

,question,sql,tables,gpt_sql_nocontext
0,What are the names of the amenities that Smith Hall has?,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',"[DORM, HAS_AMENITY, DORM_AMENITY]",SELECT Amenities.amenity FROM Amenities JOIN Dorms ON Amenities.dorm_id = Dorms.dorm_id WHERE Dorms.name = 'Smith Hall'
1,What are the names of the amenities that Smith Hall has?,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',"[DORM, HAS_AMENITY, DORM_AMENITY]",SELECT amenity FROM Amenities WHERE dorm_id = (SELECT dorm_id FROM Dorms WHERE name = 'Smith Hall')
2,How many dorms are there and what is the total capacity for each gender?,"SELECT count(*) , sum(student_capacity) , gender FROM dorm GROUP BY gender",[DORM],"SELECT COUNT(*), gender, SUM(capacity) FROM dorm GROUP BY gender"
3,Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid","[DORM, HAS_AMENITY]","SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name"
4,Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid","[DORM, HAS_AMENITY]","SELECT Dorms.dorm_name, COUNT(Amenities.amenity) AS number_of_amenities FROM Dorms LEFT JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Dorms.capacity > 100 GROUP BY Dorms.dorm_name"
...,...,...,...,...
61,How many female students live in Smith Hall?,SELECT count(*) FROM student AS T1 JOIN lives_in AS T2 ON T1.stuid = T2.stuid JOIN dorm AS T3 ON T3.dormid = T2.dormid WHERE T3.dorm_name = 'Smith Hall' AND T1.sex = 'F',"[STUDENT, LIVES_IN, DORM]",SELECT COUNT(*) FROM student WHERE gender = 'Female' AND dorm_id = (SELECT id FROM dorm WHERE name = 'Smith Hall')
62,How many female students live in Smith Hall?,SELECT count(*) FROM student AS T1 JOIN lives_in AS T2 ON T1.stuid = T2.stuid JOIN dorm AS T3 ON T3.dormid = T2.dormid WHERE T3.dorm_name = 'Smith Hall' AND T1.sex = 'F',"[STUDENT, LIVES_IN, DORM]",SELECT COUNT(*) FROM student INNER JOIN dorm ON student.dorm_id = dorm.id WHERE student.gender = 'Female' AND dorm.name = 'Smith Hall'
63,What are the names of all the dorms that don't have any amenities?,SELECT dorm_name FROM dorm WHERE dormid NOT IN (SELECT dormid FROM has_amenity),"[DORM, HAS_AMENITY]",SELECT Dorms.dorm_name FROM Dorms LEFT JOIN Amenities ON Dorms.dorm_id = Amenities.dorm_id WHERE Amenities.dorm_id IS NULL
64,Find the number of students for the cities where have more than one student.,"SELECT count(*) , city_code FROM student GROUP BY city_code HAVING count(*) > 1",[STUDENT],"SELECT city, COUNT(*) as num_students FROM student GROUP BY city HAVING COUNT(*) > 1"


# Detect all relevant tables GPT used.
Starting with the tables used by the GPT without any context.

In [92]:
tables = []
for q in df['gpt_sql_nocontext']:
    try: 
        t = Parser(q).tables
        tables.append([v.upper() for v in t])
    except:
        tables.append(None)
tables

[['AMENITIES', 'DORMS'],
 ['AMENITIES', 'DORMS'],
 ['DORM'],
 ['DORMS', 'AMENITIES'],
 ['DORMS', 'AMENITIES'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['DORM_AMENITIES'],
 ['STUDENT', 'DORM'],
 ['STUDENT', 'DORM'],
 ['DORM'],
 ['DORM'],
 ['STUDENT'],
 ['DORM'],
 ['STUDENT'],
 ['STUDENT'],
 ['DORM', 'DORM_AMENITIES'],
 ['DORM', 'DORM_AMENITIES'],
 ['STUDENT'],
 ['STUDENT', 'DORM'],
 ['STUDENT', 'DORM'],
 ['STUDENT', 'DORM'],
 ['STUDENT', 'DORM'],
 ['AMENITIES'],
 ['DORM'],
 ['STUDENT'],
 ['STUDENT', 'DORM'],
 ['STUDENT', 'DORM', 'DORM_AMENITIES'],
 ['DORM_1'],
 ['AMENITIES'],
 ['DORM', 'STUDENT'],
 ['DORM'],
 ['DORM'],
 ['STUDENT', 'DORM', 'DORM_AMENITIES'],
 ['STUDENT', 'DORM', 'DORM_AMENITIES'],
 ['STUDENT'],
 ['STUDENT'],
 ['DORM'],
 ['STUDENT'],
 ['DORM_1'],
 ['DORM_1'],
 ['DORM_1'],
 ['DORM'],
 ['DORM_INFO'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['DORM_INFO'],
 ['STUDENT'],
 ['STUDENT'],
 ['STUDENT'],
 ['DORM', 'DORM_AMENITIES'],
 

In [93]:
df['tables_referred_by_gpt_sql_nocontext'] = tables

And then detecting the tables used by the GPT with the table-schemas.


In [86]:
tables = []
cnt = 0 
for q in df_q['gpt_sql_global_schema']:
    print(q)
    cnt = cnt + 1
    try: 
        t = Parser(q).tables
        tables.append([v.upper() for v in t])
    except:
        tables.append(None)
df_q['tables_referred_by_gpt_sql_global_schema'] = tables


SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_CATALOG_SUBJECT_OFFERED GROUP BY DEPARTMENT_NAME
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE < CURRENT_DATE ORDER BY TERM_START_DATE DESC
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE > CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE <= CURRENT_DATE AND TERM_END_DATE >= CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_Courses FROM SUBJECT_OFFERED
SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PERSON.MIT_ID FROM MIT_STUDENT_DIRECTORY INNE

# Save Dataframe with generated answers


In [84]:
df_q.to_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/spiderdata_gpt_sql_v0.0.csv', index=False)
df_q

,db_id,query,question,query_toks,query_toks_no_value,question_toks,tables,gpt_sql_nocontext
0,dorm_1,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',What are the names of the amenities that Smith Hall has?,"[SELECT, T3.amenity_name, FROM, dorm, AS, T1, JOIN, has_amenity, AS, T2, ON, T1.dormid, =, T2.dormid, JOIN, dorm_amenity, AS, T3, ON, T2.amenid, =, T3.amenid, WHERE, T1.dorm_name, =, 'Smith, Hall, ']","[select, t3, ., amenity_name, from, dorm, as, t1, join, has_amenity, as, t2, on, t1, ., dormid, =, t2, ., dormid, join, dorm_amenity, as, t3, on, t2, ., amenid, =, t3, ., amenid, where, t1, ., dorm_name, =, value]","[What, are, the, names, of, the, amenities, that, Smith, Hall, has, ?]","[DORM, HAS_AMENITY, DORM_AMENITY]","[SELECT amenity FROM amenities WHERE dorm_id = (SELECT id FROM dorms WHERE name = 'Smith Hall'), SELECT a.amenity FROM amenities AS a JOIN dorms AS d ON a.dorm_id = d.id WHERE d.name = 'Smith Hall']"
1,dorm_1,"SELECT count(*) , sum(student_capacity) , gender FROM dorm GROUP BY gender",How many dorms are there and what is the total capacity for each gender?,"[SELECT, count, (, *, ), ,, sum, (, student_capacity, ), ,, gender, FROM, dorm, GROUP, BY, gender]","[select, count, (, *, ), ,, sum, (, student_capacity, ), ,, gender, from, dorm, group, by, gender]","[How, many, dorms, are, there, and, what, is, the, total, capacity, for, each, gender, ?]",[DORM],"[SELECT COUNT(DISTINCT dorm_name), gender, SUM(capacity) FROM dorm_1 GROUP BY gender]"
2,dorm_1,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid",Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"[SELECT, count, (, *, ), ,, T1.dormid, FROM, dorm, AS, T1, JOIN, has_amenity, AS, T2, ON, T1.dormid, =, T2.dormid, WHERE, T1.student_capacity, >, 100, GROUP, BY, T1.dormid]","[select, count, (, *, ), ,, t1, ., dormid, from, dorm, as, t1, join, has_amenity, as, t2, on, t1, ., dormid, =, t2, ., dormid, where, t1, ., student_capacity, >, value, group, by, t1, ., dormid]","[Find, the, number, of, amenities, for, each, of, the, dorms, that, can, accommodate, more, than, 100, students, .]","[DORM, HAS_AMENITY]","[SELECT dorms.dorm_name, COUNT(amenities.amenity) FROM dorms JOIN amenities ON dorms.dorm_id = amenities.dorm_id WHERE dorms.capacity > 100 GROUP BY dorms.dorm_name, SELECT dorms.dorm_name, COUNT(*) AS number_of_amenities FROM dorms INNER JOIN amenities ON dorms.dorm_id = amenities.dorm_id WHERE dorms.capacity > 100 GROUP BY dorms.dorm_name]"
3,dorm_1,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,What is the code of the city with the most students?,"[SELECT, city_code, FROM, student, GROUP, BY, city_code, ORDER, BY, count, (, *, ), DESC, LIMIT, 1]","[select, city_code, from, student, group, by, city_code, order, by, count, (, *, ), desc, limit, value]","[What, is, the, code, of, the, city, with, the, most, students, ?]",[STUDENT],"[SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1, SELECT city_code FROM student WHERE student_id IN (SELECT student_id FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1)]"
4,dorm_1,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,Find the code of city where most of students are living in.,"[SELECT, city_code, FROM, student, GROUP, BY, city_code, ORDER, BY, count, (, *, ), DESC, LIMIT, 1]","[select, city_code, from, student, group, by, city_code, order, by, count, (, *, ), desc, limit, value]","[Find, the, code, of, city, where, most, of, students, are, living, in, .]",[STUDENT],[SELECT city_code FROM student WHERE count(student_id) = (SELECT MAX(count(student_id)) FROM student GROUP BY city_code)]
...,...,...,...,...,...,...,...,...
45,dorm_1,"SELECT avg(age) , city_code 

# READ DATAFRAME


In [ ]:
df_q = pd.read_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/spiderdata_gpt_sql_v0.0.csv')

In [5]:
sql = []
tables = []
tables_referred_by_gpt_sql_global_schema = []
tables_referred_by_gpt_sql_nocontext = []
sql_outputsizes = []
for i in range(len(df_q)):
    sql.append(ast.literal_eval(df_q['sql'][i]))
    tables.append(ast.literal_eval(df_q['tables'][i]))
    tables_referred_by_gpt_sql_global_schema.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_global_schema'][i]))
    tables_referred_by_gpt_sql_nocontext.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_nocontext'][i]))
    sql_outputsizes.append(ast.literal_eval(df_q['sql_outputsizes'][i]))
df_q['sql'] = sql
df_q['tables'] = tables
df_q['tables_referred_by_gpt_sql_global_schema'] = tables_referred_by_gpt_sql_global_schema
df_q['tables_referred_by_gpt_sql_nocontext'] = tables_referred_by_gpt_sql_nocontext
df_q['sql_outputsizes'] = sql_outputsizes

In [95]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_q

question  \
0                                           How many organisation units in human resources are relabeled?   
1                                           How many organisation units in human resources are relabeled?   
2                               How many organisation units in human resources are new and not relabeled?   
3                                                 COUNT the number of courses offered by each department.   
4                                                       What are the past, upcoming, and current courses?   
5                                                       What are the past, upcoming, and current courses?   
6                                                       What are the past, upcoming, and current courses?   
7                                                        Which faculty has had the most location changes?   
8                                          What is the average number of courses a student can enroll in?   
9                                                                     Which students are employed by mit?   
10                                                   What is the shortest course available this semester?   
11                                                                      Show buildings and their address.   
12                                                                          How many students do we have?   
13                                                                       Show all statuses for materials?   
14                                                       What are the different cities WHERE people live?   
15                                                       What are the different cities WHERE people live?   
16                       Give the average number of courses in one department with more than 100 courses.   
17                                                               Count the number of rooms in a building.   
18                                                          Count the number of space-units per building.   
19                 For courses that have descriptions of Regular and Initial, what are their start dates?   
20                     What are the names of the 3 departments which offer the most courses per semester?   
21                     What are the names of the 3 departments which offer the most courses per semester?   
22                     What are the names of the 3 departments which offer the most courses per semester?   
23                                                           WHERE is Prof. Michael Stonebraker's office?   
24                                                                Which employees at MIT have a namesake?   
25                         Show the name for subjects which are enrollable for Computer Science Students.   
26                                                           How many students does each department have?   
27                                          What categories are there for independent activities periods?   
28                                                                  Find the city with the post code 501.   
29                                                               How many employees are there all in all?   
30                                                            How many departments does each school have?   
31               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
32               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
33               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
34                                                Show the school with the largest number of departments.   
35                                                 What are the courses with the maximum number of units?   
36                                                 What are the 

# Calculate the output-size


In [132]:
def calculate_output_size(query, tables=[]):

    if 'LIMIT' in query:
        contains_limit = True
        limit_number = re.search(r'\bLIMIT\s+(\d+)', query)
        limit_value = int(limit_number.group(1))
        query = query[:query.index('LIMIT ')]
        query = 'SELECT * FROM (' + query + ') WHERE ROWNUM <=' + str(limit_value)
    formatted_query = f"SELECT DISTINCT COUNT(*) FROM({query})"
    rows = execute_query(formatted_query, tables)
    return rows[0][0]

### Calculate outputs for expected and actual results


In [ ]:
sizes = []
for q in df_q['gpt_sql_global_schema']:
    sizes.append(calculate_output_size(q))
    print(calculate_output_size(q))
df_q['gpt_sql_global_schema_outputsizes'] = sizes
sizes = []
cnt = 0
for i in range(len(df_q)):
    size = []
    for j in range(len(df_q['sql'][i])):
        print(df_q['sql'][i][j][:-1])
        output = calculate_output_size(df_q['sql'][i][j][:-1], df_q['tables'][i])
        size.append(output)
        print(output)
    sizes.append(size)
df_q['sql_outputsizes'] = sizes

SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
1
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
2
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
3
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
62
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
62
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
58
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
71
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
0
SELECT COUNT(*) FROM

### Check how many sizes are matching and how many queries are failing


In [9]:
cnt_true = 0
cnt_neg = 0
for i in range(len(df_q)):
    if df_q['gpt_sql_global_schema_outputsizes'][i] in df_q['sql_outputsizes'][i] and df_q['gpt_sql_global_schema_outputsizes'][i]>0:
        cnt_true += 1;
    if df_q['gpt_sql_global_schema_outputsizes'][i] < 0:
        cnt_neg +=1
print(cnt_true)
print(len(df_q))
print(cnt_neg)

21
65
17


# Compare Results
Idea to base the result comparison based on the output size:
<ol>
  <li>We check at first only outputs with the equal amount of rows in their result.</li>
  <li>outputs of size 10 or less are completely computable and every element is comparable (without sorting).</li>
  <li>larger outputs are compared via sorting/hashing and comparison.</li>
  <li>Failing queries should be marked as incorrect attached with their error message.</li>
  <li>(Maybe later for fuzzy comparisons)Manual checks: queries with no output, queries with different dimensions, and duplications.</li>
</ol>

1. We check outputsize and dimensionality.

In [ ]:
correct_size = []
for i in range(len(df_q)):
    if(df_q['gpt_sql_global_schema_outputsizes'][i] not in df_q['sql_outputsizes'][i]):
        correct_size.append(0)
    else:
        correct_size.append(-1)
correct_size

In [131]:
def execute_query(query, tables):
    connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
    
    if 'LIMIT' in query:
        contains_limit = True
        limit_number = re.search(r'\bLIMIT\s+(\d+)', query)
        limit_value = int(limit_number.group(1))
        query = query[:query.index('LIMIT ')]
        query = 'SELECT * FROM (' + query + ') WHERE ROWNUM <=' + str(limit_value)

    # Set input type handler to cx_Oracle.STRING
    connection.inputtypehandler = cx_Oracle.STRING
    # Create a cursor object
    cursor = connection.cursor()
    
    # Set DEFINE OFF
    cursor.setoutputsize(0)
    
    # Execute SQL queries
    try:
        cursor.execute(query)
        # Fetch the results
        rows = cursor.fetchall()
    except Exception as e1:
        #Preprocessing wueries
        for table in tables:
            table_mod = ' WAREUSER.' + table
            query = query.replace(' ' + table, table_mod)
            print(query)
        try:
            cursor.execute(query)
            # Fetch the results
            rows = cursor.fetchall()
        except Exception as e2:
            # Print the error message to stderr
            print("An error occurred:", str(e1))
            print("An error occurred:", str(e2))
            return [[-1]]
    cursor.close()
    connection.close()
    return rows

### Start with single comparisons
Generate output and compare result

In [144]:
def compare_single_outputs(expected_queries, expected_tables, given_query, given_tables):
    expected_results = []
    for query in expected_queries:
        expected_result = execute_query(query[:-1], expected_tables)[0][0]
        expected_results.append(expected_result)
        print(expected_result)
    actual_result = execute_query(given_query, given_tables)[0][0]
    print(actual_result)
    return actual_result in expected_results

In [146]:
correct = []
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] == 1) and (1 in df_q['sql_outputsizes'][i]) and df_q['correct'][i]<0:
        print(i)
        print(df_q['sql'][i])
        print(df_q['gpt_sql_global_schema'][i])
        res = compare_single_outputs(df_q['sql'][i], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i])
        if res:
            cnt +=1
        correct.append(int(res))
        print(res)
    else:
        correct.append(df_q['correct'][i])
print(cnt)

0
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
635
635
True
1
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
635
685
False
2
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW MINUS SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
50
50
True
7
['SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT FROM FCLT_BUILDING_ADDRESS_HIST GROUP BY FCLT_BUILDING_KEY);']
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT FROM WAREUSER.

In [156]:
print(len(correct_size))
print(sum(correct_size))
df_q['gpt_sql_global_schema_outputsizes'].value_counts().get(-1, 0)

65
-25


16

### Less than 10 results
Idea: check if the sizes match and whether each result is contain in one another.

In [181]:
def compare_countable_outputs(expected_queries, expected_tables, given_query, given_tables):
    expected_results = []
    actual_result = execute_query(given_query, given_tables)
    for query in expected_queries:
        expected_result = execute_query(query[:-1], expected_tables)
        if len(expected_result)==0 and len(expected_result)==0:
            return True
        expected_results.append(expected_result)
    is_matching = []
    for i in range(len(expected_results)):
        match = True
        for j in range(len(expected_results[i])):
            if not expected_results[i][j] in actual_result:
                match= False
                break
        is_matching.append(match)
    return any(is_matching)

In [ ]:
correct_m = []
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] >=0) and (df_q['gpt_sql_global_schema_outputsizes'][i] <=10) and (df_q['correct'][i]==-1):
        res = compare_countable_outputs(df_q['sql'][i], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i])
        if res:
            cnt +=1
        correct_m.append(int(res))
    else:
        correct_m.append(df_q['correct'][i])
print(cnt)
correct_m

In [185]:
df_q['correct']=correct_m

### Larger Output sizes
Idea: check if samples from the expected result are all included in the actual result.

In [188]:
def compare_incountable_outputs(expected_comparable_query, expected_tables, given_query, given_tables, dict_res):
    expected_result = execute_query(expected_comparable_query[:-1], expected_tables)
    actual_result = execute_query(given_query, given_tables)
    if len(expected_result[0]) != len(actual_result[0]):
        return -1
    
    sorted_expected_data = sorted(expected_result, key=lambda x: x[0])
    dict_res[expected_comparable_query]=sorted_expected_data
    sorted_actual_data = sorted(actual_result, key=lambda x: x[0])
    for i in range(len(sorted_expected_data)):
        if sorted_expected_data[i] != sorted_actual_data[i]:
            return False
    return True

In [ ]:
def compare_incountable_with_precalculated(sorted_expected_data, given_query, given_tables):
    sorted_actual_data = sorted(actual_result, key=lambda x: x[0])
    for i in range(len(sorted_expected_data)):
        if sorted_expected_data[i] != sorted_actual_data[i]:
            return False
    return True

In [ ]:
correct_l = []
dict_res={}
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] >10) and (df_q['correct'][i]==-1):
        res = []
        for j in range(len(df_q['sql'][i])):
            if df_q['sql_outputsizes'][i][j] == df_q['gpt_sql_global_schema_outputsizes'][i]:
                if df_q['sql'][i][j] in dict_res.keys():
                    res.append(compare_incountable_with_precalculated(dict_res[df_q['sql'][i][j], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i]))
                else:
                    res.append(compare_incountable_outputs(df_q['sql'][i][j], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i]))
        if any(res):
            cnt +=1
        correct_l.append(int(any(res)))
    else:
        correct_l.append(df_q['correct'][i])
print(cnt)
correct_l

In [190]:
df_q['correct'] = correct_l

In [235]:
print("Total size of the data:   " + str(len(df_q)))
wrong_out = len(df_q)+sum(correct_size)
print("#wrong outputsize:        " + str(wrong_out))
print("   #wrong sqls:           " + str(df_q['gpt_sql_global_schema_outputsizes'].value_counts().get(-1, 0)))
print("#correct outputsize:      " + str(-1*sum(correct_size)))
print("   #1size-oputput:        " + str(sum((e==0 or e ==1) for e in correct)-wrong_out))
print("      #correct 1-size:    " + str(sum(e ==1 for e in correct)))
print("   #>1size-oputput:       " + str(-sum(correct_size)+wrong_out - sum((e==0 or e ==1) for e in correct)))
print("      #<11size-oputput:   " + str(sum(e==0 or e ==1 for e in correct_m)-sum(e==0 or e ==1 for e in correct)))
print("         #0size-oputput:  " + str(1))
print("         #corr<10size-out:" + str(sum(e ==1 for e in correct_m)-sum(e ==1 for e in correct)))
print("      #>10size-oputput:   " + str(sum(e==0 or e ==1 for e in correct_l)- sum(e==0 or e ==1 for e in correct_m)))
print("         #corr>10size-out:" + str(sum(e ==1 for e in correct_l)- sum(e ==1 for e in correct_m)))
print("Total #correct:           " + str(sum(df_q['correct'])))

Total size of the data:   65
#wrong outputsize:        40
   #wrong sqls:           16
#correct outputsize:      25
   #1size-oputput:        17
      #correct 1-size:    8
   #>1size-oputput:       8
      #<11size-oputput:   5
         #0size-oputput:  1
         #corr<10size-out:5
      #>10size-oputput:   3
         #corr>10size-out:3
Total #correct:           16


# Calculate Accuracy, Precision, Recall and F1-Score

Accuracy: $\frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$

Precision: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$

Recall: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$

F1-Score: $2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

In [78]:
print('Accuracy: ' + str(sum(accuracy)/len(accuracy) * 100) + '%')

Accuracy: 42.89855072463768%


In [53]:
df

,question,sql,tables,gpt_sql_nocontext,tables_referred_by_gpt_sql_nocontext
0,What are the names of the amenities that Smith Hall has?,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',"[DORM, HAS_AMENITY, DORM_AMENITY]",SELECT amenity FROM amenities WHERE dorm_id = (SELECT id FROM dorms WHERE name = 'Smith Hall'),"[AMENITIES, DORMS]"
1,What are the names of the amenities that Smith Hall has?,SELECT T3.amenity_name FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid JOIN dorm_amenity AS T3 ON T2.amenid = T3.amenid WHERE T1.dorm_name = 'Smith Hall',"[DORM, HAS_AMENITY, DORM_AMENITY]",SELECT a.amenity FROM amenities AS a JOIN dorms AS d ON a.dorm_id = d.id WHERE d.name = 'Smith Hall',"[AMENITIES, DORMS]"
2,How many dorms are there and what is the total capacity for each gender?,"SELECT count(*) , sum(student_capacity) , gender FROM dorm GROUP BY gender",[DORM],"SELECT COUNT(DISTINCT dorm_name), gender, SUM(capacity) FROM dorm_1 GROUP BY gender",[DORM_1]
3,Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid","[DORM, HAS_AMENITY]","SELECT dorms.dorm_name, COUNT(amenities.amenity) FROM dorms JOIN amenities ON dorms.dorm_id = amenities.dorm_id WHERE dorms.capacity > 100 GROUP BY dorms.dorm_name","[DORMS, AMENITIES]"
4,Find the number of amenities for each of the dorms that can accommodate more than 100 students.,"SELECT count(*) , T1.dormid FROM dorm AS T1 JOIN has_amenity AS T2 ON T1.dormid = T2.dormid WHERE T1.student_capacity > 100 GROUP BY T1.dormid","[DORM, HAS_AMENITY]","SELECT dorms.dorm_name, COUNT(*) AS number_of_amenities FROM dorms INNER JOIN amenities ON dorms.dorm_id = amenities.dorm_id WHERE dorms.capacity > 100 GROUP BY dorms.dorm_name","[DORMS, AMENITIES]"
5,What is the code of the city with the most students?,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,[STUDENT],SELECT city_code FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1,[STUDENT]
6,What is the code of the city with the most students?,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,[STUDENT],SELECT city_code FROM student WHERE student_id IN (SELECT student_id FROM student GROUP BY city_code ORDER BY COUNT(*) DESC LIMIT 1),[STUDENT]
7,Find the code of city where most of students are living in.,SELECT city_code FROM student GROUP BY city_code ORDER BY count(*) DESC LIMIT 1,[STUDENT],SELECT city_code FROM student WHERE count(student_id) = (SELECT MAX(count(student_id)) FROM student GROUP BY city_code),[STUDENT]
8,What is the last name of every student who is either female or living in a city with the code BAL or male and under 20?,SELECT lname FROM student WHERE sex = 'F' AND city_code = 'BAL' UNION SELECT lname FROM student WHERE sex = 'M' AND age < 20,[STUDENT],SELECT last_name FROM student WHERE gender = 'F' OR city_code = 'BAL' OR (gender = 'M' AND age < 20),[STUDENT]
9,Find the number of dorms that have some amenity.,SELECT count(DISTINCT dormid) FROM has_amenity,[HAS_AMENITY],SELECT COUNT(DISTINCT dorm_id) FROM dorm_1.amenities,[DORM_1.AMENITIES]


## Based on Table usage

In [94]:
accuracy = []
for t0, t1 in zip(df['tables'], df['tables_referred_by_gpt_sql_nocontext']):
    a = None 
    if t0 and t1: 
        a = 1.0*len(set(t0).intersection(set(t1)))/len(t0)
    accuracy.append(a)

In [95]:
sum(accuracy)/len(accuracy) * 100

58.28282828282827

In [96]:
df['gpt_sql_nocontext_accuracy']=accuracy 